## Event Coincidence Analysis (ECA) networks 

Event Coincidence Analysis (ECA) is a statistical method used to identify synchronous events between event time series (i.e. two locations in DOMINO-SEE), following two user-specified parameters. This approach allows us to construct networks where nodes represent extreme events in locations and edges represent significant coincidences of extreme events between locations.

This notebook demonstrates how to use the `dominosee` package to perform ECA and construct event-based climate networks from time series data.

### Input Data

We'll start by creating a synthetic dataset to demonstrate the ECA workflow. For real-world applications, you would typically load climate data from NetCDF files or other sources. In this example, we'll generate a simple dataset with the Standardized Precipitation Index (SPI) values, which are commonly used to identify drought conditions.

We'll create a synthetic dataset with the following properties:
- Spatial dimensions: 20x20 grid points (latitude x longitude)
- Temporal dimension: 365 days (daily data for one year)
- Variable: SPI1

In [1]:
import numpy as np
import xarray as xr

xr.set_options(display_expand_data=False)

In [2]:
# Using smaller dimensions for demonstration
nx_demo, ny_demo, nt_demo = 20, 20, 365

# Create coordinates
lats = np.linspace(-90, 90, nx_demo)
lons = np.linspace(-180, 180, ny_demo)
times = xr.date_range("1950-01-01", periods=nt_demo, freq="D")

# Create standard normal data for SPI values (mean=0, std=1)
spi_data = np.random.normal(0, 1, size=(nx_demo, ny_demo, nt_demo))  # Standard normal distribution

# Create xarray Dataset
spi = xr.Dataset(
    data_vars={
        "SPI1": (["lat", "lon", "time"], spi_data)
    },
    coords={
        "lat": lats,
        "lon": lons,
        "time": times
    }
)

spi

<xarray.Dataset> Size: 1MB
Dimensions:  (lat: 20, lon: 20, time: 365)
Coordinates:
  * lat      (lat) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lon      (lon) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * time     (time) datetime64[ns] 3kB 1950-01-01 1950-01-02 ... 1950-12-31
Data variables:
    SPI1     (lat, lon, time) float64 1MB -0.1615 1.393 0.208 ... 0.2356 -0.1454

### Extract the timing of extreme events

To perform Event Coincidence Analysis, we first need to identify when and where extreme events occur. In this step, we convert our continuous SPI time series into binary event time series by applying a threshold.

We'll use the `get_event` function from the `dominosee.eventorize` module, which identifies events based on a specified threshold and direction (above or below). For drought events, we're interested in SPI values below a threshold of -1.0.

In [3]:
from dominosee.eventorize import get_event
da_event = get_event(spi.SPI1, threshold=-1.0, extreme="below", event_name="drought")
da_event

<xarray.DataArray 'drought' (lat: 20, lon: 20, time: 365)> Size: 146kB
False False False False False False ... False False False False False False
Coordinates:
  * lat      (lat) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lon      (lon) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * time     (time) datetime64[ns] 3kB 1950-01-01 1950-01-02 ... 1950-12-31
Attributes:
    threshold:      -1.0
    extreme:        below
    long_name:      drought events
    description:    Events with -1.0 below threshold
    event_name:     drought
    select:         None
    select_period:  None

The result is a binary DataArray with the same dimensions as the original SPI data, where:
- `True` indicates that an extreme event (drought) occurred at that location and time
- `False` indicates no extreme event

This binary representation allows us to easily identify when and where extreme events occur throughout our dataset. The event DataArray includes attributes that document the threshold and criteria used to define the events.

### Identify number of precursor events and trigger events of ECA among location pairs

Event Coincidence Analysis examines whether events in one time series (location A) are typically followed or preceded by events in another time series (location B) within a specified time window (which is specified by the `window`, `tau` and `sym` parameters). This allows us to identify temporal relationships of extreme events between different locations.

In ECA, we distinguish between:
1. **Precursor events**: Events at location A that precede events at location B
2. **Trigger events**: Events at location A that are followed by events at location B
However, as we use a symmetrical window (by specifying `sym=True`), we do not distinguish which location has earlier events but to quantify the co-occurrence of events at both locations.

The ECA metrics quantify the strength of these relationships based on event co-occurrences. We'll use the `dominosee.eca` module to calculate these metrics.

In [4]:
from dominosee.eca import get_eca_precursor_from_events, get_eca_trigger_from_events

In [5]:
# Calculate precursor events
da_precursor = get_eca_precursor_from_events(eventA=da_event, eventB=da_event, delt=2, sym=True, tau=0)
da_precursor

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xarray.DataArray 'drought' (latA: 20, latB: 20, lonA: 20, lonB: 20)> Size: 320kB
52 34 29 31 27 21 32 29 28 35 33 31 25 ... 28 23 27 26 32 26 25 27 14 33 26 47
Coordinates:
  * latA     (latA) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonA     (lonA) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * latB     (latB) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonB     (lonB) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
Attributes:
    long_name:    Precursor Events
    units:        count
    description:  Number of precursor events (from location A to location B) ...
    eca_params:   {"delt": 2, "sym": true, "tau": 0}

In [6]:
# Calculate trigger events
da_trigger = get_eca_trigger_from_events(eventA=da_event, eventB=da_event, delt=10, sym=True, tau=0)
da_trigger

<xarray.DataArray 'drought' (latA: 20, latB: 20, lonA: 20, lonB: 20)> Size: 320kB
52 59 54 60 51 45 59 67 49 64 67 57 50 ... 52 48 53 55 67 51 55 63 49 67 57 47
Coordinates:
  * latA     (latA) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonA     (lonA) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * latB     (latB) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonB     (lonB) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
Attributes:
    long_name:    Trigger Events
    units:        count
    description:  Number of trigger events (from location A to location B) in...
    eca_params:   {"delt": 10, "sym": true, "tau": 0}

The `get_eca_precursor_from_events` function calculates the number of precursor events between all pairs of grid points. The resulting DataArray has four dimensions:
- `latA`, `lonA`: Coordinates of the location A
- `latB`, `lonB`: Coordinates of the location B

The function parameters control the ECA calculation:
- `delt`: The time window (in time steps) within which events must occur to be considered coincident
- `sym`: Whether to use a symmetric time window (before and after)
- `tau`: The required delay between events

Similarly, `get_eca_trigger_from_events` calculates the number of trigger events - events at location A that are followed by events at location B within the specified time window.

### Calculate the statistical confidence of ECA

After calculating the raw event coincidence counts, the next critical step is to determine which connections are statistically significant. This involves:

1. **Calculating confidence levels**: By assuming the random occurrence of events, we can obtain the confidence level of the event coincidence counts for both the precursor and trigger events.

2. **Setting significance thresholds**: We can determine a chosen significance level (e.g., 95% or 99%) above which coincidence counts are considered statistically significant.

In [7]:
from dominosee.eca import get_eca_precursor_confidence, get_eca_trigger_confidence

In [8]:
da_prec_conf = get_eca_precursor_confidence(precursor=da_precursor, eventA=da_event, eventB=da_event)
da_prec_conf

<xarray.DataArray 'prec_conf' (latA: 20, latB: 20, lonA: 20, lonB: 20)> Size: 640kB
1.0 0.9997 0.9985 0.9974 0.9944 0.963 ... 0.9821 0.9725 0.3567 0.9988 0.9846 1.0
Coordinates:
  * latA     (latA) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonA     (lonA) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * latB     (latB) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonB     (lonB) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
Attributes:
    long_name:    Precursor confidence
    units:        
    description:  Confidence of precursor rates (from location A to location ...
    eca_params:   {"delt": 2, "sym": true, "tau": 0}

In [9]:
da_trig_conf = get_eca_trigger_confidence(trigger=da_trigger, eventA=da_event, eventB=da_event)
da_trig_conf

<xarray.DataArray 'trigger_conf' (latA: 20, latB: 20, lonA: 20, lonB: 20)> Size: 640kB
1.0 1.0 1.0 1.0 1.0 0.9875 1.0 1.0 0.9998 ... 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
Coordinates:
  * latA     (latA) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonA     (lonA) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * latB     (latB) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonB     (lonB) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
Attributes:
    long_name:    Trigger confidence
    units:        
    description:  Confidence of trigger rates (from location A to location B)...
    eca_params:   {"delt": 10, "sym": true, "tau": 0}

### Construct network represented by adjacency matrix

The next step involves constructing the climate network based on our statistically significant connections. We'll create an adjacency matrix, which is a fundamental representation of the network structure:

- **Adjacency Matrix**: A square matrix where both rows and columns correspond to grid points (locations).
- **Matrix Elements**: Each element (i,j) contains a binary value:
  - 1 if a statistically significant connection exists between locations i and j
  - 0 if no significant connection is present
- **Interpretation**: This matrix provides a concise representation of the network topology, enabling further analysis of the climate system's spatial relationships.

In [10]:
from dominosee.network import get_link_from_confidence

In [11]:
da_link = get_link_from_confidence(da_prec_conf, 0.99) & get_link_from_confidence(da_trig_conf, 0.99)
da_link

<xarray.DataArray (latA: 20, latB: 20, lonA: 20, lonB: 20)> Size: 160kB
True True True True True False True ... True False False False True False True
Coordinates:
  * latA     (latA) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonA     (lonA) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0
  * latB     (latB) float64 160B -90.0 -80.53 -71.05 -61.58 ... 71.05 80.53 90.0
  * lonB     (lonB) float64 160B -180.0 -161.1 -142.1 ... 142.1 161.1 180.0

In [12]:
print(f"The network density is {da_link.sum().values/da_link.size*100:.2f}%.")

The network density is 41.58%.
